# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import patsy
import numpy as np
import statsmodels.api as sm
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split


import statsmodels.formula.api as smf
from patsy import dmatrix

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

### Tratando os dados faltantes

In [4]:
# Retirando  registros com tempo emprego null
df.dropna(subset = ['tempo_emprego'], inplace = True)
#Verificando se não há mais dados nulos
df.isna().sum()

Unnamed: 0               0
data_ref                 0
id_cliente               0
sexo                     0
posse_de_veiculo         0
posse_de_imovel          0
qtd_filhos               0
tipo_renda               0
educacao                 0
estado_civil             0
tipo_residencia          0
idade                    0
tempo_emprego            0
qt_pessoas_residencia    0
renda                    0
dtype: int64

### Separando a base em treinamento e teste (25% para teste, 75% para treinamento)

#### Excluindo variáveis desnecessárias

In [5]:
df.columns

Index(['Unnamed: 0', 'data_ref', 'id_cliente', 'sexo', 'posse_de_veiculo',
       'posse_de_imovel', 'qtd_filhos', 'tipo_renda', 'educacao',
       'estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego',
       'qt_pessoas_residencia', 'renda'],
      dtype='object')

In [6]:
df.drop(['Unnamed: 0', 'data_ref', 'id_cliente'],axis=1, inplace = True)
df.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [7]:
df.columns

Index(['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos',
       'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia', 'idade',
       'tempo_emprego', 'qt_pessoas_residencia', 'renda'],
      dtype='object')

#### Separando a base em treinamento e teste (25% para teste, 75% para treinamento)

In [8]:
y=df.renda
x=df.drop ('renda', axis =1)
x_treino,x_teste,y_treino,y_teste = train_test_split(x,y,test_size =0.25, random_state = 10 )

In [9]:
print('Registros em x:',x.shape[0])
print('Registros em y:',y.shape[0])
print('---------------------')
print('Registros em y_teste:',y_teste.shape[0])
print('Registros em x_teste:',x_teste.shape[0])
print('---------------------')
print('Registros em x_treino:',x_treino.shape[0])
print('Registros em y_treino:',y_treino.shape[0])

Registros em x: 12427
Registros em y: 12427
---------------------
Registros em y_teste: 3107
Registros em x_teste: 3107
---------------------
Registros em x_treino: 9320
Registros em y_treino: 9320


### Rodando uma regularização ridge com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avaliação do R2 na base de testes

In [10]:
#Unindo Y_treino e X_treino  para usar no smf.ols
df_treino = pd.DataFrame(y_treino).merge(x_treino, left_index=True, right_index= True , how='left')
df_treino

,renda,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia
5269,6014.68,F,False,True,0,Empresário,Secundário,Casado,Casa,40,15.109589,2.0
4685,1634.01,M,True,False,1,Assalariado,Secundário,Casado,Casa,26,0.380822,3.0
820,7692.33,M,True,True,0,Assalariado,Secundário,Casado,Com os pais,44,8.523288,2.0
4703,1935.13,M,False,True,0,Empresário,Superior completo,Solteiro,Aluguel,27,3.345205,1.0
572,2552.31,F,False,False,0,Assalariado,Secundário,Casado,Casa,52,9.030137,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8784,2298.63,F,False,True,2,Assalariado,Secundário,Solteiro,Casa,33,10.186301,3.0
14006,581.38,F,False,True,0,Empresário,Superior completo,Casado,Casa,47,3.293151,2.0
1617,2420.39,F,False,True,1,Empresário,Secundário,Separado,Casa,42,0.263014,2.0
8786,18227.85,F,False,False,0,Assalariado,Secundário,Viúvo,Casa,56,15.443836,1.0


In [11]:
#Criando uma lista com uma regressão para cada alpha

alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1] 
regressoes = []

for alpha in alphas: 

    modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia' 
    md = smf.ols(modelo, data = df_treino)
    regressoes.append( md.fit_regularized(method = 'elastic_net' 
                             , refit = True 
                             , L1_wt = 0 #0 para Ridge
                             , alpha = alpha) )

In [12]:
#Calculando R2 na base de teste

#Calculando R2 de cada regressão e adicionando à R2_list
R2_list=[]

for reg in regressoes:

    #criando dataframe para  calculo da correlação do y_Teste x y_teste_predito
    df_y_teste_predito = pd.DataFrame(reg.predict(x_teste))
    df_y_teste_real = pd.DataFrame(y_teste)
    df_corr = pd.DataFrame(df_y_teste_real).merge(df_y_teste_predito, left_index=True, right_index= True , how='left')
    #Calculo da Correlação entre  y_Teste x y_teste_predito
    r = df_corr.corr().iloc[0,1]
    #Calculo do R² entre  y_Teste x y_teste_predito
    R2 = r**2
    R2_list.append(R2)

In [13]:
# Criando um DataFrame comparando os R2 de cada modelo

dic_R2 = { 'Modelos': [] ,
          'Regularização': [],
           'Alpha':   [],
           'R2' :     [],
         }
i= 0

for reg in regressoes:
    i_string= str(i+1)                       
    dic_R2['Modelos'].append('Modelo '+i_string)
    dic_R2['Alpha'].append(alphas[i])
    dic_R2['Regularização'].append('Ridge')
    dic_R2['R2'].append(R2_list[i])  
    i=i+1 
    
df_ridge = pd.DataFrame(dic_R2)
df_ridge 

,Modelos,Regularização,Alpha,R2
0,Modelo 1,Ridge,0.000,0.262212
1,Modelo 2,Ridge,0.001,0.262414
2,Modelo 3,Ridge,0.005,0.262637
3,Modelo 4,Ridge,0.010,0.262753
4,Modelo 5,Ridge,0.050,0.261024
5,Modelo 6,Ridge,0.100,0.256046


O modelo 4 apresenta R² ligeiramente maior que os demais modelos.

### Rodando uma regularização LASSO com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avaliação do R2 na base de testes

In [14]:
#Criando uma lista com uma regressão LASSO para cada alpha

alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1] 
regressoes = []

for alpha in alphas: 

    modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia' 
    md = smf.ols(modelo, data = df_treino)
    regressoes.append( md.fit_regularized(method = 'elastic_net' 
                             , refit = True 
                             , L1_wt = 1 #1 para LASSO
                             , alpha = alpha) )

In [15]:
#Calculando R2 na base de teste

#Calculando R2 de cada regressão e adicionando a R2_list
R2_list=[]

for reg in regressoes:

    #criando dataframe para  calculo da correlação do y_Teste x y_teste_predito
    df_y_teste_predito = pd.DataFrame(reg.predict(x_teste))
    df_y_teste_real = pd.DataFrame(y_teste)
    df_corr = pd.DataFrame(df_y_teste_real).merge(df_y_teste_predito, left_index=True, right_index= True , how='left')
    #Calculo da Correlação entre  y_Teste x y_teste_predito
    r = df_corr.corr().iloc[0,1]
    #Calculo do R² entre  y_Teste x y_teste_predito
    R2 = r**2
    R2_list.append(R2)

In [16]:
# Criando um DataFrame comparando os R2 de cada modelo

dic_R2 = { 'Modelos': [] ,
          'Regularização': [],
           'Alpha':   [],
           'R2' :     [],
         }
i= 0

for reg in regressoes:
    i_string= str(i+7)                       
    dic_R2['Modelos'].append('Modelo '+i_string)
    dic_R2['Alpha'].append(alphas[i])
    dic_R2['Regularização'].append('Lasso')
    dic_R2['R2'].append(R2_list[i])  
    i=i+1 
    
df_lasso = pd.DataFrame(dic_R2)


In [17]:
# Juntando os dataframes modelos
df_comparativo = pd.concat([df_ridge,df_lasso])
df_comparativo

,Modelos,Regularização,Alpha,R2
0,Modelo 1,Ridge,0.000,0.262212
1,Modelo 2,Ridge,0.001,0.262414
2,Modelo 3,Ridge,0.005,0.262637
3,Modelo 4,Ridge,0.010,0.262753
4,Modelo 5,Ridge,0.050,0.261024
5,Modelo 6,Ridge,0.100,0.256046
0,Modelo 7,Lasso,0.000,0.262212
1,Modelo 8,Lasso,0.001,0.262212
2,Modelo 9,Lasso,0.005,0.262212
3,Modelo 10,Lasso,0.010,0.262212


O Modelo 4 permaneceu com o melhor coeficiente de determinaçção R2

### Rodando um modelo stepwise e avaliação de R2 na base de teste

O stepwise não aceita variáveis categóricas portanto devemos converte-las em dummies

In [19]:
df_treino.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9320 entries, 5269 to 1551
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   renda                  9320 non-null   float64
 1   sexo                   9320 non-null   object 
 2   posse_de_veiculo       9320 non-null   bool   
 3   posse_de_imovel        9320 non-null   bool   
 4   qtd_filhos             9320 non-null   int64  
 5   tipo_renda             9320 non-null   object 
 6   educacao               9320 non-null   object 
 7   estado_civil           9320 non-null   object 
 8   tipo_residencia        9320 non-null   object 
 9   idade                  9320 non-null   int64  
 10  tempo_emprego          9320 non-null   float64
 11  qt_pessoas_residencia  9320 non-null   float64
dtypes: bool(2), float64(3), int64(2), object(5)
memory usage: 1.1+ MB


In [20]:
df_treino['renda']

5269      6014.68
4685      1634.01
820       7692.33
4703      1935.13
572       2552.31
           ...   
8784      2298.63
14006      581.38
1617      2420.39
8786     18227.85
1551      1001.98
Name: renda, Length: 9320, dtype: float64

In [21]:
#usando patsy para converter a matriz em dummies
y, x = patsy.dmatrices(modelo, data = df_treino)
#Vamos transformar x em um dataframe pois o parâmetro de entrada do stepwise deve ser um dataframe
df_x_treino = pd.DataFrame(x)
df_x_treino.columns = x.design_info.column_names #renomeia as colunas do dataframe com o nome das colunas da designmatrix
df_x_treino.head()

,Intercept,sexo[T.M],posse_de_veiculo[T.True],posse_de_imovel[T.True],tipo_renda[T.Bolsista],tipo_renda[T.Empresário],tipo_renda[T.Pensionista],tipo_renda[T.Servidor público],educacao[T.Pós graduação],educacao[T.Secundário],...,estado_civil[T.Viúvo],tipo_residencia[T.Casa],tipo_residencia[T.Com os pais],tipo_residencia[T.Comunitário],tipo_residencia[T.Estúdio],tipo_residencia[T.Governamental],qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia
0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,40.0,15.109589,2.0
1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,26.0,0.380822,3.0
2,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,44.0,8.523288,2.0
3,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,3.345205,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,52.0,9.030137,2.0


In [22]:
df_x_treino.shape

(9320, 25)

In [23]:
y.shape

(9320, 1)

In [24]:
X = df_x_treino
y = y

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

variaveis = stepwise_selection(X, y)

print('resulting features:')
print(variaveis)

Add  Intercept                      with p-value 0.0
#############
['Intercept']
Add  tempo_emprego                  with p-value 0.0
#############
['Intercept', 'tempo_emprego']
Add  sexo[T.M]                      with p-value 1.21252e-254
#############
['Intercept', 'tempo_emprego', 'sexo[T.M]']
Add  idade                          with p-value 4.15928e-08
#############
['Intercept', 'tempo_emprego', 'sexo[T.M]', 'idade']
Add  tipo_renda[T.Empresário]       with p-value 4.47018e-06
#############
['Intercept', 'tempo_emprego', 'sexo[T.M]', 'idade', 'tipo_renda[T.Empresário]']
Add  educacao[T.Superior completo]  with p-value 0.000265837
#############
['Intercept', 'tempo_emprego', 'sexo[T.M]', 'idade', 'tipo_renda[T.Empresário]', 'educacao[T.Superior completo]']
Add  qt_pessoas_residencia          with p-value 0.00442174
#############
['Intercept', 'tempo_emprego', 'sexo[T.M]', 'idade', 'tipo_renda[T.Empresário]', 'educacao[T.Superior completo]', 'qt_pessoas_residencia']
#############
[

Stepwise escolheu as seguintes variáveis:

'tempo_emprego', 'sexo[T.M]', 'idade', 'Intercept', 'tipo_renda[T.Empresário]', 'educacao[T.Superior completo]', 'qt_pessoas_residencia'

#### Acertando o dataframe para excluir as variáveis não escolhidas pelo stepwise

In [25]:
df_treino.loc[ df_treino['educacao'] == 'Primário', 'educacao2'] ='Outro'
df_treino.loc[ df_treino['educacao'] == 'Secundário', 'educacao2'] = 'Outro'
df_treino.loc[ df_treino['educacao'] == 'Superior completo', 'educacao2'] = 'Superior completo'
df_treino.loc[ df_treino['educacao'] == 'Superior incompleto', 'educacao2'] = 'Outro'
df_treino.loc[ df_treino['educacao'] == 'Pós graduação', 'educacao2'] = 'Outro'

df_treino.loc[ df_treino['tipo_renda'] == 'Empresário', 'tipo_renda2'] ='Empresário'
df_treino.loc[ df_treino['tipo_renda'] == 'Assalariado', 'tipo_renda2'] = 'Outro'
df_treino.loc[ df_treino['tipo_renda'] == 'Servidor público', 'tipo_renda2'] = 'Outro'
df_treino.loc[ df_treino['tipo_renda'] == 'Pensionista', 'tipo_renda2'] = 'Outro'
df_treino.loc[ df_treino['tipo_renda'] == 'Bolsista', 'tipo_renda2'] = 'Outro'

df_treino[['educacao', 'educacao2','tipo_renda','tipo_renda2']].head()

,educacao,educacao2,tipo_renda,tipo_renda2
5269,Secundário,Outro,Empresário,Empresário
4685,Secundário,Outro,Assalariado,Outro
820,Secundário,Outro,Assalariado,Outro
4703,Superior completo,Superior completo,Empresário,Empresário
572,Secundário,Outro,Assalariado,Outro


In [26]:
y, x = patsy.dmatrices('renda ~ C(sexo,Treatment(0)) +  idade + C(educacao2)+ C(tipo_renda2, Treatment(1)) + qt_pessoas_residencia + tempo_emprego', data = df_treino)
x

DesignMatrix with shape (9320, 7)
  Columns:
    ['Intercept',
     'C(sexo, Treatment(0))[T.M]',
     'C(educacao2)[T.Superior completo]',
     'C(tipo_renda2, Treatment(1))[T.Empresário]',
     'idade',
     'qt_pessoas_residencia',
     'tempo_emprego']
  Terms:
    'Intercept' (column 0)
    'C(sexo, Treatment(0))' (column 1)
    'C(educacao2)' (column 2)
    'C(tipo_renda2, Treatment(1))' (column 3)
    'idade' (column 4)
    'qt_pessoas_residencia' (column 5)
    'tempo_emprego' (column 6)
  (to view full data, use np.asarray(this_obj))

In [27]:
#Rodando a  regressão e avaliando os parâmetros

reg= sm.OLS(y, x).fit()
reg.summary()
   

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.252
Model:                            OLS   Adj. R-squared:                  0.252
Method:                 Least Squares   F-statistic:                     524.2
Date:                Tue, 02 Jan 2024   Prob (F-statistic):               0.00
Time:                        12:41:19   Log-Likelihood:                -96467.
No. Observations:                9320   AIC:                         1.929e+05
Df Residuals:                    9313   BIC:                         1.930e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================================================
                                                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
Intercept                                  -3710.8527    478.641     -7.753      0.000   -4649.094   -2772.611
C(sexo, Treatment(0))[T.M]                  5949.6926    166.640     35.704      0.000    5623.042    6276.343
C(educacao2)[T.Superior completo]            588.0926    162.868      3.611      0.000     268.835     907.350
C(tipo_renda2, Treatment(1))[T.Empresário]   798.4435    176.367      4.527      0.000     452.725    1144.162
idade                                         59.2384      9.185      6.449      0.000      41.233      77.244
qt_pessoas_residencia                        247.0473     86.772      2.847      0.004      76.955     417.139
tempo_emprego                                539.1555     12.440     43.342      0.000     514.771     563.540
==============================================================================
Omnibus:                    13695.949   Durbin-Watson:                   2.014
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          9803605.007
Skew:                           8.672   Prob(JB):                         0.00
Kurtosis:                     160.938   Cond. No.                         263.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [28]:
#Calculando R2 do modelo com stepwise na base de teste

#criando dataframe para  calculo da correlação do y_Teste x y_teste_predito
df_y_teste_predito = pd.DataFrame(reg.predict(x))
df_y_teste_real = pd.DataFrame(y)
df_corr = pd.DataFrame(df_y_teste_real).merge(df_y_teste_predito, left_index=True, right_index= True , how='left')
#Calculo da Correlação entre  y_Teste x y_teste_predito
r = df_corr.corr().iloc[0,1]
#Calculo do R² entre  y_Teste x y_teste_predito
R2 = r**2
R2

0.25246885559707816

In [29]:
R2 = reg.rsquared
df = pd.DataFrame({ 'Modelos': ['Modelo 13'],'Regularização': ['Stepwise'],'Alpha':  ['---'],'R2' :[R2],})
df

,Modelos,Regularização,Alpha,R2
0,Modelo 13,Stepwise,---,0.252469


In [30]:
df_comparativo

,Modelos,Regularização,Alpha,R2
0,Modelo 1,Ridge,0.000,0.262212
1,Modelo 2,Ridge,0.001,0.262414
2,Modelo 3,Ridge,0.005,0.262637
3,Modelo 4,Ridge,0.010,0.262753
4,Modelo 5,Ridge,0.050,0.261024
5,Modelo 6,Ridge,0.100,0.256046
0,Modelo 7,Lasso,0.000,0.262212
1,Modelo 8,Lasso,0.001,0.262212
2,Modelo 9,Lasso,0.005,0.262212
3,Modelo 10,Lasso,0.010,0.262212


O melhor modelo, segundo o critério do coeficiente de determinação R² é o modelo 4, contudo, o modelo apresenta todas as variáveis o que o deixa com uma complexidade muito grande. Já o modelo usando Stepwise embora tem um R² ligeiramente menor que os demais, ele é bem menos complexo com poucas variáveis e pode ser a melhor escolha. Vamos tentar melhorá-lo com algumas transformações.

### Melhorando o modelo Stepwise com log na variável resposta.

In [31]:
y, x = patsy.dmatrices('np.log(renda) ~ C(sexo,Treatment(0)) +  idade + C(educacao2)+ C(tipo_renda2, Treatment(1)) + qt_pessoas_residencia + tempo_emprego', data = df_treino)
x
   

DesignMatrix with shape (9320, 7)
  Columns:
    ['Intercept',
     'C(sexo, Treatment(0))[T.M]',
     'C(educacao2)[T.Superior completo]',
     'C(tipo_renda2, Treatment(1))[T.Empresário]',
     'idade',
     'qt_pessoas_residencia',
     'tempo_emprego']
  Terms:
    'Intercept' (column 0)
    'C(sexo, Treatment(0))' (column 1)
    'C(educacao2)' (column 2)
    'C(tipo_renda2, Treatment(1))' (column 3)
    'idade' (column 4)
    'qt_pessoas_residencia' (column 5)
    'tempo_emprego' (column 6)
  (to view full data, use np.asarray(this_obj))

In [32]:
#Rodando a  regressão e avaliando os parâmetros

reg= sm.OLS(y, x).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.351
Model:                            OLS   Adj. R-squared:                  0.351
Method:                 Least Squares   F-statistic:                     840.0
Date:                Tue, 02 Jan 2024   Prob (F-statistic):               0.00
Time:                        12:44:15   Log-Likelihood:                -10153.
No. Observations:                9320   AIC:                         2.032e+04
Df Residuals:                    9313   BIC:                         2.037e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================================================
                                                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
Intercept                                      7.0595      0.045    155.168      0.000       6.970       7.149
C(sexo, Treatment(0))[T.M]                     0.7908      0.016     49.928      0.000       0.760       0.822
C(educacao2)[T.Superior completo]              0.1121      0.015      7.239      0.000       0.082       0.142
C(tipo_renda2, Treatment(1))[T.Empresário]     0.1449      0.017      8.643      0.000       0.112       0.178
idade                                          0.0072      0.001      8.223      0.000       0.005       0.009
qt_pessoas_residencia                          0.0302      0.008      3.663      0.000       0.014       0.046
tempo_emprego                                  0.0606      0.001     51.278      0.000       0.058       0.063
==============================================================================
Omnibus:                        0.713   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.700   Jarque-Bera (JB):                0.683
Skew:                           0.018   Prob(JB):                        0.711
Kurtosis:                       3.022   Cond. No.                         263.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [33]:
#calculando R² do modelo Stepwise com log na variável resposta para a base de teste
# montando o df para calcular a matriz correlação
df_y = pd.DataFrame(np.exp(reg.fittedvalues), columns = ['Y Predito']) # utilizando np.exp para 'tirar' o log 
df_y['Y'] = pd.DataFrame(y)
#calculando R² a partir da correlação entre Y e Y Predito
r = df_y [['Y', 'Y Predito']].corr().iloc[0,1]
R2 = r**2
R2

0.31556134849315526

In [34]:
# montando um df comparativo
df = pd.DataFrame({ 'Modelos': ['Modelo 13 com log em y'],'Regularização': ['Stepwise'],'Alpha':  ['---'],'R2' :[R2]})
df_comparativo = pd.concat([df_comparativo,df])
df_comparativo

,Modelos,Regularização,Alpha,R2
0,Modelo 1,Ridge,0.0,0.262212
1,Modelo 2,Ridge,0.001,0.262414
2,Modelo 3,Ridge,0.005,0.262637
3,Modelo 4,Ridge,0.01,0.262753
4,Modelo 5,Ridge,0.05,0.261024
5,Modelo 6,Ridge,0.1,0.256046
0,Modelo 7,Lasso,0.0,0.262212
1,Modelo 8,Lasso,0.001,0.262212
2,Modelo 9,Lasso,0.005,0.262212
3,Modelo 10,Lasso,0.01,0.262212


O modelo 13 com log na variável resposta obteve significativa melhora no coeficiente de determinação R², superando os demais, além disso é o menos complexo (menor número de variáveis), sendo agora o melhor modelo encontrado.

### Ajustando uma árvore de regressão e avaliando R²

In [35]:
reg = DecisionTreeRegressor(random_state = 10)
reg.fit(df_x_treino, y_treino)

DecisionTreeRegressor(random_state=10)

In [36]:
#Unindo Y_teste e X_teste  para usar no score e calcular R2 na base de teste
df_teste = pd.DataFrame(y_teste).merge(x_teste, left_index=True, right_index= True , how='left')
df_teste

,renda,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia
14904,6088.93,M,False,False,1,Empresário,Primário,Casado,Casa,29,6.802740,3.0
2644,1779.36,F,False,True,0,Assalariado,Secundário,Casado,Casa,40,3.435616,2.0
6363,3101.24,F,False,False,1,Assalariado,Secundário,União,Casa,33,7.526027,3.0
11222,2173.29,M,True,True,2,Assalariado,Secundário,Casado,Casa,27,5.041096,4.0
14552,5518.90,F,True,False,1,Assalariado,Superior completo,Solteiro,Casa,30,3.487671,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13897,4155.67,F,False,True,1,Assalariado,Secundário,Casado,Casa,44,11.895890,3.0
4158,1201.44,F,True,True,0,Assalariado,Superior completo,Casado,Casa,31,6.345205,2.0
986,10929.30,M,False,False,0,Assalariado,Secundário,Casado,Casa,52,13.608219,2.0
3105,5403.62,M,False,True,1,Assalariado,Secundário,União,Governamental,36,2.805479,3.0


In [37]:
#usando patsy para converter x_teste em dummies
y_teste, x_teste = patsy.dmatrices(modelo, data = df_teste)
#Vamos transformar x em um dataframe pois o parâmetro de entrada do stepwise deve ser um dataframe
df_x_teste = pd.DataFrame(x_teste)
df_x_teste.columns = x_teste.design_info.column_names #renomeia as colunas do dataframe com o nome das colunas da designmatrix
#df_x_teste.head()
df_x_teste

,Intercept,sexo[T.M],posse_de_veiculo[T.True],posse_de_imovel[T.True],tipo_renda[T.Bolsista],tipo_renda[T.Empresário],tipo_renda[T.Pensionista],tipo_renda[T.Servidor público],educacao[T.Pós graduação],educacao[T.Secundário],...,estado_civil[T.Viúvo],tipo_residencia[T.Casa],tipo_residencia[T.Com os pais],tipo_residencia[T.Comunitário],tipo_residencia[T.Estúdio],tipo_residencia[T.Governamental],qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia
0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,29.0,6.802740,3.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,40.0,3.435616,2.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,33.0,7.526027,3.0
3,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,2.0,27.0,5.041096,4.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,30.0,3.487671,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3102,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,44.0,11.895890,3.0
3103,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,31.0,6.345205,2.0
3104,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,52.0,13.608219,2.0
3105,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,36.0,2.805479,3.0


In [38]:
#Calculando R² na base de teste
R2 = reg.score(df_x_teste, y_teste)
R2

0.32600718139297535

In [39]:
# montando um df comparativo
df = pd.DataFrame({ 'Modelos': ['Modelo 14 Árvore de Regressão'],'Regularização': ['---'],'Alpha':  ['---'],'R2' :[R2]})
df_comparativo = pd.concat([df_comparativo,df])
df_comparativo

,Modelos,Regularização,Alpha,R2
0,Modelo 1,Ridge,0.0,0.262212
1,Modelo 2,Ridge,0.001,0.262414
2,Modelo 3,Ridge,0.005,0.262637
3,Modelo 4,Ridge,0.01,0.262753
4,Modelo 5,Ridge,0.05,0.261024
5,Modelo 6,Ridge,0.1,0.256046
0,Modelo 7,Lasso,0.0,0.262212
1,Modelo 8,Lasso,0.001,0.262212
2,Modelo 9,Lasso,0.005,0.262212
3,Modelo 10,Lasso,0.01,0.262212


### CONCLUSÃO

De todos os modelos estudados a árvore de regressão (Modelo 14) teve o melhor coeficiente de determinação R²